# 라마로 테스트

In [1]:
# #ollama 서버 실행중인지 확인
# from langchain_community.chat_models import ChatOllama
# 
# llm = ChatOllama(model="llama3:latest")
# llm.invoke("서울이란?")

AIMessage(content='😊\n\n"서울" (Seoul) is the capital city of South Korea. It is a megacity with a population of over 10 million people, making it one of the most populous cities in the world.\n\nSeoul is known for its rich history and culture, dating back to the 18th century when it was the capital of the Joseon Dynasty. Today, it is a modern city with a vibrant economy, cutting-edge technology, and a blend of traditional and contemporary architecture.\n\nSome popular attractions in Seoul include:\n\n1. Gyeongbokgung Palace: The largest and most iconic palace in Korea.\n2. Bukchon Hanok Village: A traditional Korean village with preserved hanok houses.\n3. Myeong-dong Shopping District: One of the busiest shopping areas in Asia.\n4. Namsan Tower (Seoul Tower): A popular spot for panoramic views of the city.\n5. Insadong Street: A famous street filled with traditional tea houses, shops, and restaurants.\n\nSeoul is also known for its food scene, with popular dishes like bibimbap, bulgogi

In [2]:
# #prompt template
# from langchain_core.prompts import ChatPromptTemplate
# 
# prompt = ChatPromptTemplate.from_messages([
#     ("system", "You are a helpful, professional assistant named 권봇. Introduce yourself first, and answer the questions. answer me in Korean no matter what. "),
#     ("user", "{input}")
# ])
# 
# chain = prompt | llm
# chain.invoke({"input": "What is stock??"})

AIMessage(content="😊\n\n스톡(stock)입니다. STOCK는 회사의 주식이나 자본을 의미합니다. 즉, 특정 회사에 대한 지분이나 권리를 나타내는 증서입니다. 일반적으로 투자자들은股票을 구입하여 해당 회사의 성장으로부터 이익을 얻습니다. STOCK는 주로 주식시장에서 교환되며, 이를 통해 투자자들은 다양한 회사의 주식을 구입하거나 판매할 수 있습니다.\n\n(Translation: Stock is a company's stock or capital. It means the share or right to a specific company. Generally, investors buy stocks and gain benefits from the growth of that company. Stocks are traded on the stock market, allowing investors to purchase or sell shares of various companies.)", response_metadata={'model': 'llama3:latest', 'created_at': '2024-07-18T10:07:59.1804134Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 1530939500, 'load_duration': 3622800, 'prompt_eval_count': 48, 'prompt_eval_duration': 47954000, 'eval_count': 157, 'eval_duration': 1477965000}, id='run-f3a4a1ea-b004-4644-97cc-a8560e688add-0')

In [3]:
# #output parser사용
# from langchain_core.output_parsers import StrOutputParser
# 
# chain = prompt | llm | StrOutputParser()
# chain.invoke({"input": "What is stock?"})

'😊\n\n스톡(stock)란 기업의 주식 또는 자본을 의미합니다. 즉, 특정 기업이 발행한 주식을 말하는 것입니다. 이 주식은 투자자들이 기업의 성장 가능성을 예상하여 구입하거나 판매할 수 있는 증권입니다.'

In [4]:
# #체인 스트림 출력
# chain = prompt | llm | StrOutputParser()
# for token in chain.stream(
#     {"input": "What is stock?"}
# ):
#     print(token, end="")

😊

 stocks, or, , are shares of ownership in a company that represent a claim on part of its assets and profits. In other words, stocks give you a small piece of a company's equity, allowing you to participate in its growth and success.

Stocks can be traded on stock exchanges like the Korea Exchange (KRX) or over-the-counter markets, providing investors with an opportunity to buy and sell shares in various companies.

Do you have any other questions about stocks?

In [5]:
# # 첫번째 체인
# prompt1 = ChatPromptTemplate.from_template("[{korean_input}] translate the question into English. Don't say anything else, just translate it.")
# chain1 = (
#     prompt1 
#     | llm 
#     | StrOutputParser()
# )
# message1 = chain1.invoke({"korean_input": "주식이 뭐야?"})
# print(f'message1: {message1}') # What is a stock?
# 
# # 두번째 체인
# prompt2 = ChatPromptTemplate.from_messages([
#     ("system", "You are a helpful, professional assistant named 권봇. answer the question in Korean"),
#     ("user", "{input}")
# ])
# chain2 = (
#     {"input": chain1}
#     | prompt2
#     | llm
#     | StrOutputParser()
# )
# message2 = chain2.invoke({"korean_input": "주식이 뭐야?"})
# print(f'message2: {message2}') # 주식은 한 회사의 소유권을 나타내는 증권입니다. 즉, 특정 기업에 투자하여 (중략)

message1: "What is stock?"
message2: 😊

식권(stock)은 일정한 조건에 따라 발행된 주식을 의미합니다. 주식은 기업의 자산을 구성하는 부분 중 하나로, 주주가 지분을 가질 수 있는 증서를 말합니다. 주식은 주당 가격으로 매매되며, 주주는 해당 주식의 소유권을 갖게 됩니다.

(Translation: Stock refers to a certain type of security that is issued under specific conditions. Stocks are part of a company's assets and represent ownership stakes in the business. Stock prices are traded on an exchange, and shareholders own the corresponding stocks.)


In [6]:
# #체인 병렬 실행
# from langchain_core.runnables import RunnableParallel
# joke_chain = (
#     ChatPromptTemplate.from_template("{topic}에 관련해서 짧은 농담 말해줘") 
#     | llm)
# poem_chain = (
#     ChatPromptTemplate.from_template("{topic}에 관련해서 시 2줄 써줘") 
#     | llm)
# 
# # map_chain = {"joke": joke_chain, "poem": poem_chain} # 체인에서 이처럼 사용할 때, 자동으로 RunnableParallel 사용됨
# # map_chain = RunnableParallel({"joke": joke_chain, "poem": poem_chain})
# map_chain = RunnableParallel(joke=joke_chain, poem=poem_chain)
# 
# map_chain.invoke({"topic": "애플"})

{'joke': AIMessage(content="Here's a short Apple-related joke:\n\nWhy did the Apple employee break up with his girlfriend?\n\nBecause he needed more space... and a better battery life! 🤣🍎", response_metadata={'model': 'llama3:latest', 'created_at': '2024-07-18T10:08:15.8757367Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 1078639400, 'load_duration': 1520900, 'prompt_eval_count': 14, 'prompt_eval_duration': 14051000, 'eval_count': 38, 'eval_duration': 336061000}, id='run-0856d868-7623-43a6-817f-e44dcc257b0f-0'),
 'poem': AIMessage(content="Here are two lines about Apple:\n\nInnovative designs and user-friendly interfaces have made Apple a household name, with the iPhone revolutionizing the way we communicate. From Macs to iPads, Apple's ecosystem has become an integral part of modern life, shaping the tech industry for decades to come.", response_metadata={'model': 'llama3:latest', 'created_at': '2024-07-18T10:08:15.5197738Z'

# GPT 3.5-turbo로 테스트

In [2]:
from dotenv import load_dotenv
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableLambda

# 환경 변수 로드
load_dotenv()

# OpenAI API 키 설정
OPENAI_API_KEY = "OPENAI_API_KEY"

# ChatOpenAI 모델 초기화
chat = ChatOpenAI(model="gpt-4", temperature=0.1)

# 프롬프트 템플릿 설정
template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            텍스트 기반 어드벤처 게임의 내레이터 역할을 하세요. 당신의 임무는 환경과 보조 캐릭터를 묘사하는 것입니다. 보조 캐릭터가 말할 때는 직접 화법을 사용하세요. 플레이어가 플레이어 캐릭터(PC)의 행동과 대사를 조종합니다. 플레이어 캐릭터를 대신해 행동하거나 말해서는 안 됩니다. 내레이터가 상황을 묘사하고 플레이어가 플레이어 캐릭터의 행동을 말하는 식으로 턴이 진행됩니다. 플레이어 캐릭터에 대해 이야기할 때는 2인칭 시점을 사용하세요. 당신의 출력은 베스트셀러 작가가 쓴 것처럼 전문적으로 작성되어야 합니다.
            -------
                        
            {setting_info}
            
            {character_info}
            
            {current_situation}
            """,
        ),
        ("human", "{question}"),
    ]
)

# 설정 정보
setting_info = """
세계관: 스팀펑크와 마법이 공존하는 빅토리아 시대 스타일의 세계
주요 도시: 네블린 - 거대한 시계탑과 증기 기관이 가득한 산업화된 도시
정치 체제: 마법사 의회와 기술자 길드가 공동으로 통치
주요 갈등: 마법과 기술의 균형을 둘러싼 긴장 관계
특이사항: 하늘을 나는 증기선, 마법으로 작동하는 자동인형, 차원 간 여행이 가능한 포털
"""

# 캐릭터 정보
character_info = """
이름: 에이든 스파크
나이: 27세
직업: 마법공학자 견습생
외모: 붉은 곱슬머리, 녹색 눈, 항상 고글을 착용
성격: 호기심 많고 모험을 좋아하며, 약간 서툴지만 열정적
특기: 마법과 기계를 결합한 발명품 제작
목표: 마법과 기술을 완벽하게 융합한 획기적인 발명품 만들기
소지품: 다용도 마법 렌치, 증기력 소형 비행 장치
"""

# 현재 상황
current_situation = """
당신은 네블린 시의 중심부, 거대한 시계탑 아래에 있는 붐비는 시장에 서 있습니다. 증기 기관의 윙윙거리는 소리와 마법 부적들이 짤랑거리는 소리가 공기를 가득 채우고 있습니다. 갑자기 하늘에서 이상한 폭발음이 들리고, 사람들이 놀라 위를 올려다봅니다. 무언가가 빠르게 떨어지고 있는 것 같습니다. 시장 광장에 있는 사람들은 혼란에 빠져 흩어지기 시작합니다. 당신의 발명품을 시연하러 가는 중이었지만, 이 갑작스러운 상황에 어떻게 대처해야 할지 고민하고 있습니다.
"""

# RunnableLambda를 사용하여 텍스트 전달
temp_chain = {
    "setting_info": RunnableLambda(lambda _: setting_info),
    "character_info": RunnableLambda(lambda _: character_info),
    "current_situation": RunnableLambda(lambda _: current_situation),
    "question": RunnableLambda(lambda _: "폭발음을 조사한다.")
} | template | chat

# 체인 실행
response = temp_chain.invoke({})
print(response)


C:\Users\mmc\miniconda3\envs\trpg\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


content='당신은 시장 광장을 빠져나와 폭발음이 들려온 방향으로 달려갑니다. 거리를 따라 달리면서, 당신은 먼 거리에서 불길이 치솟는 것을 볼 수 있습니다. 그곳에 도착하면, 당신은 놀라운 장면을 목격합니다. 하늘에서 떨어진 것은 바로 하늘을 나는 증기선이었습니다. 그것은 건물의 지붕을 뚫고 불길 속에 빠져 있었습니다.\n\n"저런, 저 증기선은 \'하늘의 눈물\'이라는 이름의 유명한 증기선이야. 어떻게 저런 일이..." 당신 옆에서 한 남자가 속삭이듯 말합니다. 그의 얼굴은 놀라움과 슬픔으로 일그러져 있습니다.' response_metadata={'token_usage': {'completion_tokens': 282, 'prompt_tokens': 966, 'total_tokens': 1248}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-129b8099-31ab-43ed-9646-f1841e2895c3-0'


In [9]:
from langchain.prompts.chat import HumanMessagePromptTemplate, SystemMessagePromptTemplate


prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("{instructions}"),
    HumanMessagePromptTemplate.from_template("{text}")
])

instructions = """
텍스트 기반 어드벤처 게임의 내레이터 역할을 하세요. 당신의 임무는 환경과 보조 캐릭터를 묘사하는 것입니다. 보조 캐릭터가 말할 때는 직접 화법을 사용하세요. 플레이어가 플레이어 캐릭터(PC)의 행동과 대사를 조종합니다. 플레이어 캐릭터를 대신해 행동하거나 말해서는 안 됩니다. 내레이터가 상황을 묘사하고 플레이어가 플레이어 캐릭터의 행동을 말하는 식으로 턴이 진행됩니다. 플레이어 캐릭터에 대해 이야기할 때는 2인칭 시점을 사용하세요. 당신의 출력은 베스트셀러 작가가 쓴 것처럼 전문적으로 작성되어야 합니다.
"""

setting_info="""
세계관: 스팀펑크와 마법이 공존하는 빅토리아 시대 스타일의 세계
주요 도시: 네블린 - 거대한 시계탑과 증기 기관이 가득한 산업화된 도시
정치 체제: 마법사 의회와 기술자 길드가 공동으로 통치
주요 갈등: 마법과 기술의 균형을 둘러싼 긴장 관계
특이사항: 하늘을 나는 증기선, 마법으로 작동하는 자동인형, 차원 간 여행이 가능한 포털
"""

character_info = """
이름: 에이든 스파크
나이: 27세
직업: 마법공학자 견습생
외모: 붉은 곱슬머리, 녹색 눈, 항상 고글을 착용
성격: 호기심 많고 모험을 좋아하며, 약간 서툴지만 열정적
특기: 마법과 기계를 결합한 발명품 제작
목표: 마법과 기술을 완벽하게 융합한 획기적인 발명품 만들기
소지품: 다용도 마법 렌치, 증기력 소형 비행 장치
"""

current_situation="""
당신은 네블린 시의 중심부, 거대한 시계탑 아래에 있는 붐비는 시장에 서 있습니다. 증기 기관의 윙윙거리는 소리와 마법 부적들이 짤랑거리는 소리가 공기를 가득 채우고 있습니다. 갑자기 하늘에서 이상한 폭발음이 들리고, 사람들이 놀라 위를 올려다봅니다. 무언가가 빠르게 떨어지고 있는 것 같습니다. 시장 광장에 있는 사람들은 혼란에 빠져 흩어지기 시작합니다. 당신의 발명품을 시연하러 가는 중이었지만, 이 갑작스러운 상황에 어떻게 대처해야 할지 고민하고 있습니다.
"""

query_system = instructions

query = f"""
# Instructions
{instructions}

# Setting
{setting_info}

# # Character
{character_info}

# Currently
## Scene
{current_situation}

Narrator:
    """
OPENAI_API_KEY = "OPENAI_API_KEY"

story_model = ChatOpenAI(model="gpt-4",)

response = story_model(prompt.format_prompt(instructions=query_system, text=query).to_messages(), stop=["Player:"])

ValidationError: 1 validation error for SystemMessagePromptTemplate
prompt
  Can't instantiate abstract class StringPromptTemplate with abstract method format (type=type_error)

In [23]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage, AIMessage

# 지침 메시지
instructions = """
텍스트 기반 어드벤처 게임의 내레이터 역할을 하세요. 당신의 임무는 환경과 보조 캐릭터를 묘사하는 것입니다. 보조 캐릭터가 말할 때는 직접 화법을 사용하세요. 플레이어가 플레이어 캐릭터(PC)의 행동과 대사를 조종합니다. 플레이어 캐릭터를 대신해 행동하거나 말해서는 안 됩니다. 내레이터가 상황을 묘사하고 플레이어가 플레이어 캐릭터의 행동을 말하는 식으로 턴이 진행됩니다. 플레이어 캐릭터에 대해 이야기할 때는 2인칭 시점을 사용하세요. 당신의 출력은 베스트셀러 작가가 쓴 것처럼 전문적으로 작성되어야 합니다.
"""

# 설정 정보
setting_info = """
세계관: 스팀펑크와 마법이 공존하는 빅토리아 시대 스타일의 세계
주요 도시: 네블린 - 거대한 시계탑과 증기 기관이 가득한 산업화된 도시
정치 체제: 마법사 의회와 기술자 길드가 공동으로 통치
주요 갈등: 마법과 기술의 균형을 둘러싼 긴장 관계
특이사항: 하늘을 나는 증기선, 마법으로 작동하는 자동인형, 차원 간 여행이 가능한 포털
"""

# 캐릭터 정보
character_info = """
이름: 에이든 스파크
나이: 27세
직업: 마법공학자 견습생
외모: 붉은 곱슬머리, 녹색 눈, 항상 고글을 착용
성격: 호기심 많고 모험을 좋아하며, 약간 서툴지만 열정적
특기: 마법과 기계를 결합한 발명품 제작
목표: 마법과 기술을 완벽하게 융합한 획기적인 발명품 만들기
소지품: 다용도 마법 렌치, 증기력 소형 비행 장치
"""

# 현재 상황
current_situation = """
당신은 네블린 시의 중심부, 거대한 시계탑 아래에 있는 붐비는 시장에 서 있습니다. 증기 기관의 윙윙거리는 소리와 마법 부적들이 짤랑거리는 소리가 공기를 가득 채우고 있습니다. 갑자기 하늘에서 이상한 폭발음이 들리고, 사람들이 놀라 위를 올려다봅니다. 무언가가 빠르게 떨어지고 있는 것 같습니다. 시장 광장에 있는 사람들은 혼란에 빠져 흩어지기 시작합니다. 당신의 발명품을 시연하러 가는 중이었지만, 이 갑작스러운 상황에 어떻게 대처해야 할지 고민하고 있습니다.
"""

# ChatOpenAI 모델을 초기화합니다.
story_model = ChatOpenAI(model="gpt-4", openai_api_key="YOUR_OPENAI_API_KEY")

# 메시지를 생성합니다.
messages = [
    SystemMessage(content=instructions),
    HumanMessage(content=f"""
    # Setting
    {setting_info}

    # Character
    {character_info}

    # Currently
    ## Scene
    {current_situation}

    Narrator:
    """)
]

# 모델에 메시지를 입력하고 응답을 받습니다.
response = story_model.generate(messages)

# 응답을 출력합니다.
print(response.generations[0].message.content)


TypeError: Got unknown type ('content', '\n텍스트 기반 어드벤처 게임의 내레이터 역할을 하세요. 당신의 임무는 환경과 보조 캐릭터를 묘사하는 것입니다. 보조 캐릭터가 말할 때는 직접 화법을 사용하세요. 플레이어가 플레이어 캐릭터(PC)의 행동과 대사를 조종합니다. 플레이어 캐릭터를 대신해 행동하거나 말해서는 안 됩니다. 내레이터가 상황을 묘사하고 플레이어가 플레이어 캐릭터의 행동을 말하는 식으로 턴이 진행됩니다. 플레이어 캐릭터에 대해 이야기할 때는 2인칭 시점을 사용하세요. 당신의 출력은 베스트셀러 작가가 쓴 것처럼 전문적으로 작성되어야 합니다.\n')